In [ ]:
import pandas as pd
import re

In [ ]:
kapt_excel_path = '../input/20251107_단지_기본정보.xlsx'
kapt_csv_path = '../input/1_kapt.csv'

건축물대장_excel_path = '../input/표제부-총괄표제부-병합및정제-디자인추가-25.xlsx'
건축물대장_csv_path = '../input/4_건축물대장.csv'

In [ ]:
# kapt = pd.read_excel(kapt_excel_path)
# kapt.to_csv(kapt_csv_path)

# 건축물대장 = pd.read_excel(건축물대장_excel_path)
# 건축물대장.to_csv(건축물대장_csv_path)

In [ ]:
kapt = pd.read_csv(kapt_csv_path)
kapt = kapt[kapt['시도']=='서울특별시']
print(f"kapt: {len(kapt)}")

건축물대장 = pd.read_csv(건축물대장_csv_path)
print(f"건축물대장: {len(건축물대장)}")

kapt.insert(0, 'id', range(1, len(kapt) + 1))
건축물대장.insert(0, 'id', range(1, len(건축물대장) + 1))

# 법정동 주소 파싱

In [ ]:
def kapt_parse_법정동주소(addr):
    if addr is None or pd.isna(addr): return(pd.NA, pd.NA, pd.NA, pd.NA)
    addr = addr.strip()
    if not addr: return(pd.NA, pd.NA, pd.NA, pd.NA)
    
    시도 = pd.NA
    군구 = pd.NA
    동리= pd.NA
    번지= pd.NA
    상세= pd.NA
    중복주소= pd.NA
    addr_rest = addr
    
    match_groups = re.match(r"(.+?시)\s*(.*)", addr)
    if match_groups:
        시도 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
    
    match_groups = re.match(r"(.+?구)\s*(.*)", addr_rest)
    if match_groups:
        군구 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\S+)\s*(.*)", addr_rest)
    if match_groups:
        동리 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\d+(?:-\d+)?)(?:번지)?-?\s*(.*)", addr_rest)
    if match_groups:
        번지 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    if addr_rest:
        if addr_rest.startswith(','):
            중복주소 = addr_rest.lstrip(',').strip()
        else:
            상세 = addr_rest
        
    return (시도, 군구, 동리, 번지, 상세, 중복주소)

kapt[
    ['법정동주소(시도)', '법정동주소(군구)', '법정동주소(동리)','법정동주소(번지)', '법정동주소(상세)', '법정동주소(중복주소)']
] = kapt['법정동주소'].apply(
    lambda x: pd.Series(kapt_parse_법정동주소(x))
)   

In [ ]:
def 건축물대장_parse_대지위치(addr):
    if addr is None or pd.isna(addr): return(pd.NA, pd.NA, pd.NA, pd.NA)
    addr = addr.strip()
    if not addr: return(pd.NA, pd.NA, pd.NA, pd.NA)
    
    시도 = pd.NA
    군구 = pd.NA
    동리= pd.NA
    번지= pd.NA
    상세= pd.NA
    addr_rest = addr
    
    match_groups = re.match(r"(.+?시)\s*(.*)", addr)
    if match_groups:
        시도 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
    
    match_groups = re.match(r"(.+?구)\s*(.*)", addr_rest)
    if match_groups:
        군구 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\S+)\s*(.*)", addr_rest)
    if match_groups:
        동리 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\d+(?:-\d+)?)(?:번지)?-?\s*(.*)", addr_rest)
    if match_groups:
        번지 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    if addr_rest:
        if addr_rest.startswith(','):
            중복주소 = addr_rest.lstrip(',').strip()
        else:
            상세 = addr_rest
        
    return (시도, 군구, 동리, 번지, 상세)

건축물대장[
    ['대지위치(시도)', '대지위치(군구)', '대지위치(동리)','대지위치(번지)', '대지위치(상세)']
] = 건축물대장['대지위치'].apply(
    lambda x: pd.Series(건축물대장_parse_대지위치(x))
)   

# 도로명 주소 파싱

In [ ]:
def kapt_parse_도로명주소(addr):
    if addr is None or pd.isna(addr): return(pd.NA, pd.NA, pd.NA, pd.NA)
    addr = addr.strip()
    if not addr: return(pd.NA, pd.NA, pd.NA, pd.NA)
    
    시도 = pd.NA
    군구 = pd.NA
    도로명 = pd.NA
    건물번호 = pd.NA
    중복주소 = pd.NA
    상세 = pd.NA
    addr_rest = addr
    
    match_groups = re.match(r"(.+?시)\s*(.*)", addr)
    if match_groups:
        시도 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
    
    match_groups = re.match(r"(.+?구)\s*(.*)", addr_rest)
    if match_groups:
        군구 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\S+(?:로|길))\s*(.*)", addr_rest)
    if match_groups:
        도로명 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\d+(?:-\d+)?)\s*(.*)", addr_rest)
    if match_groups:
        건물번호 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
    
    if addr_rest:
        if addr_rest.startswith(','):
            중복주소 = addr_rest.lstrip(',').strip()
        else:
            상세 = addr_rest
        
    return (시도, 군구, 도로명, 건물번호, 상세, 중복주소)

kapt[
    ['도로명주소(시도)', '도로명주소(군구)', '도로명주소(도로명)', '도로명주소(건물번호)', '도로명주소(상세)', '도로명주소(중복주소)']
] = kapt['도로명주소'].apply(
    lambda x: pd.Series(kapt_parse_도로명주소(x))
)

In [ ]:
def 건축물대장_parse_도로명대지위치(addr):
    if addr is None or pd.isna(addr): return(pd.NA, pd.NA, pd.NA, pd.NA)
    addr = addr.strip()
    if not addr: return(pd.NA, pd.NA, pd.NA, pd.NA)
    
    시도 = pd.NA
    군구 = pd.NA
    도로명 = pd.NA
    건물번호 = pd.NA
    중복주소 = pd.NA
    상세 = pd.NA
    addr_rest = addr
    
    match_groups = re.match(r"(.+?시)\s*(.*)", addr)
    if match_groups:
        시도 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
    
    match_groups = re.match(r"(.+?구)\s*(.*)", addr_rest)
    if match_groups:
        군구 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\S+(?:로|길))\s*(.*)", addr_rest)
    if match_groups:
        도로명 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\d+(?:-\d+)?)\s*(.*)", addr_rest)
    if match_groups:
        건물번호 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
    
    addr_rest = re.sub(r"\([^)]*\)", "", addr_rest).strip()
    if addr_rest:
        if addr_rest.startswith(','):
            중복주소 = addr_rest.lstrip(',').strip()
        else:
            상세 = addr_rest
        
    return (시도, 군구, 도로명, 건물번호, 상세)

건축물대장[
    ['도로명대지위치(시도)', '도로명대지위치(군구)', '도로명대지위치(도로명)', '도로명대지위치(건물번호)', '도로명대지위치(상세)']
] = 건축물대장['도로명대지위치'].apply(
    lambda x: pd.Series(건축물대장_parse_도로명대지위치(x))
)

In [ ]:
print(kapt.columns)